# Exam 2 Practical

### Instructions: replace 'YOUR CODE HERE' comment with your code as needed to achieve the results described below.  Add import statements as needed.  Please use Python 3.

### Give yourself enough time so that you can 'Restart and run all' before submitting your work.  Submit a .ipynb file and, if possible, a .html file.

### There are 15 questions, with numbers indicated below.  Each question is worth 4 points.

In [3]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import tree, linear_model
from sklearn.decomposition import PCA

## Part 1.  Dimensionality Reduction

### Read the red wine dataset with Pandas

In [5]:
input_file = "https://raw.githubusercontent.com/grbruns/cst495/master/winequality-white.csv"
df = pd.read_csv(input_file, sep=";")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


#### Get the target values as a numpy array

In [7]:
y = df['quality'].astype(float).values
df.drop('quality', axis=1, inplace=True)

#### Remember the feature names; then put feature data into a numpy matrix

In [8]:
feature_names = df.columns
X = df.values
X.shape

(4898, 11)

#### 1. Perform linear regression on the data, and print the RMSE obtained on a test set.  You will have to split the data into training and test sets as a first step.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)
print(rmse)

0.5549102375097094


### Principal Component Analysis

#### Scale the data before performing PCA

In [10]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

#### 2. Apply PCA to reduce the number of dimensions of the data.  Reduce enough so that 90% of the variance in the original data is preserved.

In [11]:
pca = PCA(n_components = 0.90)
X_reduced = pca.fit_transform(X)

#### 3. Print the number of dimensions in the reduced dataset

In [12]:
print(X_reduced.shape[1])

8


#### 4. Each principal component is a weighted combination of the original features.  For the first principal component just found, what is the coefficient of the first original feature?  Print your answer using your PCA result.

In [13]:
print(pca.components_[0])

[ 0.15721845  0.00508949  0.14404984  0.42740837  0.21201106  0.30033439
  0.4066522   0.5115236  -0.12883188  0.04337933 -0.43723783]


#### 5. Compute the RMSE for linear regression with the reduced dataset.  You can copy previous code if you want.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced,y)
regr = LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
rmse = mean_squared_error(y_test, y_pred)
print(rmse)

0.6456266072112732


# Part 2. Deep Neural Nets

In [15]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

Following the tip from Jose Mijangos, you may want to uncomment the second line below if you are running TensorFlow on the GPU of your laptop.  This may help if you are getting 'ResourceExhausedError' message.  Alternatively, you can restart your kernel.

In [16]:
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

### Simple network for MNIST data

#### Read the MNIST data

In [17]:
# get MNIST data
mnist = input_data.read_data_sets("/tmp/data")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


#### Graph construction

In [18]:
tf.reset_default_graph()

# constants
n_inputs  = 28*28    # number of pixels in an MNIST image
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10     # 10 classes

# create placeholders
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# create network
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits =  tf.layers.dense(hidden2, n_outputs, name="outputs")
    
# cost function (cross entropy)
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
# select optimizer
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
# specify how to evaluate model performance
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
# create a node to initialize all variables
init = tf.global_variables_initializer()

#### Graph execution

In [19]:
n_epochs = 5   # set very low
batch_size = 50

# train the model
with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_test  = accuracy.eval(feed_dict={X: mnist.test.images,
                                             y: mnist.test.labels})
        print(epoch, "Test accuracy:", acc_test)

0 Test accuracy: 0.9024
1 Test accuracy: 0.9232
2 Test accuracy: 0.9305
3 Test accuracy: 0.9371
4 Test accuracy: 0.9437


#### 6. Copy the graph construction section, and change the graph by adding a hidden layer.  The sizes of the hidden layers should be 200, 100, 50 (200 for first hidden layer).

In [20]:
tf.reset_default_graph()

# constants
n_inputs  = 28*28    # number of pixels in an MNIST image
n_hidden1 = 200
n_hidden2 = 100
n_hidden3 = 50
n_outputs = 10     # 10 classes

# create placeholders
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# create network
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    logits =  tf.layers.dense(hidden2, n_outputs, name="outputs")
    
# cost function (cross entropy)
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
# select optimizer
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
# specify how to evaluate model performance
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
# create a node to initialize all variables
init = tf.global_variables_initializer()

#### 7. Execute the new graph.  To do this, copy the graph execution code, making changes if required.  (Variables n_epochs and batch_size can reused from earlier cells.)

In [21]:
n_epochs = 5   # set very low
batch_size = 50

# train the model
with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_test  = accuracy.eval(feed_dict={X: mnist.test.images,
                                             y: mnist.test.labels})
        print(epoch, "Test accuracy:", acc_test)

0 Test accuracy: 0.8981
1 Test accuracy: 0.9206
2 Test accuracy: 0.9284
3 Test accuracy: 0.9339
4 Test accuracy: 0.9363


#### 8. Modify the network again, this time changing to a momentum optimizer.  Note that you will need to provide a value for the 'momentum' parameter.

In [22]:
tf.reset_default_graph()

# constants
n_inputs  = 28*28    # number of pixels in an MNIST image
n_hidden1 = 200
n_hidden2 = 100
n_hidden3 = 50
n_outputs = 10     # 10 classes

# create placeholders
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# create network
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    logits =  tf.layers.dense(hidden2, n_outputs, name="outputs")
    
# cost function (cross entropy)
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
# select optimizer
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9, use_nesterov=True)
    training_op = optimizer.minimize(loss)
    
# specify how to evaluate model performance
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
# create a node to initialize all variables
init = tf.global_variables_initializer()

#### 9. Execute the new graph.  To do this, copy the graph execution code, making changes if required.

In [23]:
n_epochs = 5   # set very low
batch_size = 50

# train the model
with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_test  = accuracy.eval(feed_dict={X: mnist.test.images,
                                             y: mnist.test.labels})
        print(epoch, "Test accuracy:", acc_test)

0 Test accuracy: 0.947
1 Test accuracy: 0.9644
2 Test accuracy: 0.9706
3 Test accuracy: 0.9737
4 Test accuracy: 0.9739


#### 10. Modify the graph again, this time replacing the ReLU activation function with an ELU activation function.  As before, copy and modify the graph construction code.

In [44]:
tf.reset_default_graph()

# constants
n_inputs  = 28*28    # number of pixels in an MNIST image
n_hidden1 = 200
n_hidden2 = 100
n_hidden3 = 50
n_outputs = 10     # 10 classes

# create placeholders
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# create network
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.elu)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.elu)
    logits =  tf.layers.dense(hidden2, n_outputs, name="outputs")
    
# cost function (cross entropy)
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
# select optimizer
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9, use_nesterov=True)
    training_op = optimizer.minimize(loss)
    
# specify how to evaluate model performance
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
# create a node to initialize all variables
init = tf.global_variables_initializer()

#### 11. Execute the new graph.  To do this, copy the graph execution code, making changes if required.

In [25]:
n_epochs = 5   # set very low
batch_size = 50

# train the model
with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_test  = accuracy.eval(feed_dict={X: mnist.test.images,
                                             y: mnist.test.labels})
        print(epoch, "Test accuracy:", acc_test)

0 Test accuracy: 0.9345
1 Test accuracy: 0.9533
2 Test accuracy: 0.9622
3 Test accuracy: 0.9675
4 Test accuracy: 0.9697


#### 12. Copy and modify the graph execution code again.  This time, change the number of epochs to 10, and print the value of the loss tensor every epoch.  Loss should be evaluated with respect to the most recent mini-batch. 

In [47]:
n_epochs = 10   # set very low
batch_size = 50

# train the model
with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            cross_entropy = -tf.reduce_sum(y_batch * tf.log(y_batch))
            loss_val = sess.run(cross_entropy, feed_dict={X: X_batch, y_: y_batch})
            print("Loss: ", loss_val)
        acc_test  = accuracy.eval(feed_dict={X: mnist.test.images,
                                             y: mnist.test.labels})
        print(epoch, "Test accuracy:", acc_test)

TypeError: Value passed to parameter 'x' has DataType uint8 not in list of allowed values: bfloat16, float16, float32, float64, complex64, complex128

# Part 3. Convolutional Neural Nets

#### A basic CNN for MNIST, based on code in our text.

Constants related to input and output size

In [27]:
height = 28
width = 28
channels = 1
n_inputs = height * width
n_outputs = 10

Graph construction

In [28]:
tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    
conv1_fmaps = 16
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")

conv2_fmaps = 32
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

pool3_fmaps = conv2_fmaps
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

n_fc1 = 64
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

Graph execution.  The number of epochs and batch size are small to speed execution.

In [29]:
n_epochs = 3
batch_size = 10

with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train accuracy:", acc_train)

0 Train accuracy: 1.0
1 Train accuracy: 1.0
2 Train accuracy: 1.0


#### 13. Print the shape of 'conv1' and 'conv2'.  Use one or two print statements.

In [30]:
print("conv1: ",conv1.get_shape())
print("conv2: ",conv2.get_shape())

conv1:  (?, 28, 28, 16)
conv2:  (?, 14, 14, 32)


#### 14. Copy the graph construction section into the cell below, and change the graph by adding a max pooling layer between the convolutional layers.  Use stride 2 and 2 x 2 kernel size.  You may need to make adjustments to later layers because of this change.

In [41]:
tf.reset_default_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    
conv1_fmaps = 16
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")

pool1_fmaps = conv1_fmaps
with tf.name_scope("pool1"):
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool1_flat = tf.reshape(pool1, shape=[-1, pool1_fmaps * 7 * 7])

conv2_fmaps = 32
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"
conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

pool3_fmaps = conv2_fmaps
with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])

n_fc1 = 64
with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

(?, 14, 14, 16)


#### 15. Execute the new graph.  To do this, copy the graph execution code, making changes if required.

In [42]:
n_epochs = 3
batch_size = 10

with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train accuracy:", acc_train)

InvalidArgumentError: Input to reshape is a tensor with 1440 values, but the requested shape requires a multiple of 784
	 [[{{node pool3/Reshape}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](pool3/MaxPool-1-0-TransposeNCHWToNHWC-LayoutOptimizer, pool3/Reshape/shape)]]

Caused by op 'pool3/Reshape', defined at:
  File "D:\Programs\Anaconda\envs\tensorflow_gpuenv\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Programs\Anaconda\envs\tensorflow_gpuenv\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Programs\Anaconda\envs\tensorflow_gpuenv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Programs\Anaconda\envs\tensorflow_gpuenv\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\Programs\Anaconda\envs\tensorflow_gpuenv\lib\site-packages\ipykernel\kernelapp.py", line 

#### 16. (Extra credit)  Copy the simple feedforward network for NMIST (not the CNN), add dropout, and execute the new graph.  Run for 10 epochs.

In [ ]:
# YOUR CODE HERE